<a href="https://colab.research.google.com/github/aporshnev/python_pipelines/blob/main/pipeline_for_poswise_frequency_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пайплайн, в начале которого происходит автоматическое определение языка, на котором написан переданный текст, далее текст токенизируется, лемматизируется, происходит первичный морфологический анализ и даже определяется, является ли это слово знаменательным. Полученные такой обработкой данные автоматически сортируются и отбираются в зависимости от запрашиваемых частеречных тэгов. 

0. Скачиваем и импортируем библиотеку для определения языка, библиотеку для нлп, пандас для удобного формата данных и функцию reduce для написания красивого кода.

In [ ]:
pip install langdetect -q

     |████████████████████████████████| 981 kB 5.4 MB/s 


In [ ]:
pip install spacy==3.0.0 -q

     |████████████████████████████████| 12.7 MB 224 kB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 9.1 MB 4.6 MB/s 
     |████████████████████████████████| 456 kB 55.8 MB/s 
     |████████████████████████████████| 623 kB 49.7 MB/s 


In [ ]:
! python -m spacy download ru_core_news_sm -q
! python -m spacy download en_core_web_sm -q

     |████████████████████████████████| 17.9 MB 138 kB/s 
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 8.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
     |████████████████████████████████| 13.7 MB 69 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from functools import reduce

import pandas as pd
from langdetect import detect
import spacy

1. Собственно, вот сами составляющие пайплайна. 

In [ ]:
def aggregate_info(text):
    '''
    Creates a df containing info about token, lemma, pos,
    and whether this word is a stopword
    '''
    # detect language ad find appropriate model
    lang = detect(text)
    if lang == 'en':
        model = 'en_core_web_sm'
    elif lang == 'ru':
        model = 'ru_core_news_sm'
    nlp = spacy.load(model)


    def compose_dict(d, token):
        '''
        Fills dict with features of new tokens
        '''
        d['token'].append(token.text)
        d['lemma'].append(token.lemma_)
        d['pos'].append(token.pos_)
        d['stop'].append(token.is_stop)
        return d

    d = {
        'token' : [],
        'lemma' : [],
        'pos': [],
        'stop': []
    }
    
    # decapitalize all the words
    text = ' '.join(map(lambda x: x.lower(), text.split()))


    return pd.DataFrame(reduce(compose_dict, nlp(text), d))

def form_freq_df(word_freq: pd.DataFrame) -> pd.DataFrame:
    '''
    Forms df in which a row is a unique token 
    for which the number of occurences is specified
    '''
    word_freq = pd.DataFrame(aggregate_info(text).value_counts()).reset_index()
    word_freq.columns = ['token', 'lemma', 'pos', 'stop', 'count']
    return word_freq


def get_top_freq_words(data: pd.DataFrame, pos: str) -> pd.DataFrame:
    '''
    sorts a df by pos and number of occurences
    '''
    return data[data.pos == pos].sort_values(by='count', ascending=False)

1.1 Предлагаю работать с началом первой главы "Гарри Поттер и принц полукровка"
Кладем текст в переменную text и начинаем удивительное путешествие

In [ ]:
text = '''It was nearing midnight and the Prime Minister was sitting alone in his office, reading a long
memo that was slipping through his brain without leaving the slightest trace of meaning behind. He
was waiting for a call from the President of a far distant country, and between wondering when the
wretched man would telephone, and trying to suppress unpleasant memories of what had been a
very long, tiring, and difficult week, there was not much space in his head for anything else. The
more he attempted to focus on the print on the page before him, the more clearly the Prime
Minister could see the gloating face of one of his political opponents. This particular opponent had
appeared on the news that very day, not only to enumerate all the terrible things that had happened
in the last week (as though anyone needed reminding) but also to explain why each and every one of
them was the government’s fault.
The Prime Minister’s pulse quickened at the very thought of these accusations, for they were
neither fair nor true. How on earth was his government supposed to have stopped that bridge
collapsing? It was outrageous for anybody to suggest that they were not spending enough on bridges.
The bridge was fewer than ten years old, and the best experts were at a loss to explain why it had
snapped cleanly in two, sending a dozen cars into the watery depths of the river below. And how
dare anyone suggest that it was lack of policemen that had resulted in those two very nasty and wellpublicized murders? Or that the government should have somehow foreseen the freak hurricane in
the West Country that had caused so much damage to both people and property? And was it his
fault that one of his Junior Ministers, Herbert Chorley, had chosen this week to act so peculiarly that
he was now going to be spending a lot more time with his family? 
“A grim mood has gripped the country,” the opponent had concluded, barely concealing his own
broad grin.
And unfortunately, this was perfectly true. The Prime Minister felt it himself; people really did
seem more miserable than usual. Even the weather was dismal; all this chilly mist in the middle of
July…It wasn’t right, it wasn’t normal… '''

Собираем общий анализ слов в тексте, выглядит это так

In [ ]:
analysis = aggregate_info(text)
analysis.head()

,token,lemma,pos,stop
0,it,it,PRON,True
1,was,be,AUX,True
2,nearing,near,VERB,False
3,midnight,midnight,NOUN,False
4,and,and,CCONJ,True


Формируем из этого частотный словарь, выглядит так

In [ ]:
word_freq = form_freq_df(analysis)
word_freq.head()

,token,lemma,pos,stop,count
0,the,the,DET,True,28
1,",",",",PUNCT,False,17
2,was,be,AUX,True,15
3,of,of,ADP,True,11
4,and,and,CCONJ,True,11


И при помощи такой удобной функции можно выбирать, слова какой части речи мы хотим искать 

In [ ]:
top = get_top_freq_words(word_freq, 'NOUN')
top.head()

,token,lemma,pos,stop,count
18,minister,minister,NOUN,False,4
24,week,week,NOUN,False,3
26,country,country,NOUN,False,3
23,government,government,NOUN,False,3
32,opponent,opponent,NOUN,False,2


Вот такие у нас топ10 глаголов, наречий и прилагательных 

In [ ]:
print(get_top_freq_words(word_freq, 'VERB').head(10)['token'].to_list())
print(get_top_freq_words(word_freq, 'ADV').head(10)['token'].to_list())
print(get_top_freq_words(word_freq, 'ADJ').head(10)['token'].to_list())

['suggest', 'were', 'spending', 'explain', 'wellpublicized', 'stopped', 'snapped', 'slipping', 'sitting', 'sending']
['more', 'how', 'very', 'why', 'so', 'enough', 'peculiarly', 'perfectly', 'really', 'now']
['prime', 'much', 'true', 'very', 'long', 'more', 'watery', 'usual', 'unpleasant', 'normal']


1.2 Теперь то же, но для русского. Текст берем тот же самый, но в профессиональном переводе.

In [ ]:
text = '''Приближалась полночь. Премьер-министр си­дел у себя в кабинете в полном одиночестве и читал длинный меморандум. Строчки мелькали перед гла­зами, не задевая сознания. Премьер-министр ожи­дал звонка от президента одной далекой страны. Он раздумывал, когда же наконец позвонит этот зло­счастный тип, и одновременно пытался отделаться от неприятных воспоминаний о необычайно дол­гой и утомительной неделе; ни на что другое в го­лове у него просто не оставалось места. Чем больше он старался сосредоточиться на печатной страни­це, которая лежала перед ним на столе, тем отчет­ливее видел перед собой злорадное лицо одного из своих политических противников. Не далее как се­годня противник этот, выступая в программе но­востей, не только перечислил все ужасные проис­шествия минувшей недели (как будто кому-то тре­бовалось об этом напоминать), но еще и подробно объяснил, почему в каждом из них виновато пра­вительство. У премьер-министра зачастил пульс от одной мыс­ли об этих подлых и несправедливых обвинениях. Интересно, каким это образом правительство мог­ло помешать мосту обрушиться? Возмутительная нелепость — намекать, будто на строительство мос­тов тратится недостаточно средств. Мосту не было еще и десяти лет, лучшие эксперты теряются в до­гадках, отчего он вдруг разломился ровно посере­дине, отправив дюжину автомобилей на дно реки. И как только наглости хватило заявить, что причи­на двух зверских убийств, широко освещавшихся в средствах массовой информации, — нехватка по­лицейских? И что правительство обязано было ка­ким-то образом предвидеть внезапный ураган, про­несшийся по нескольким графствам к юго-западу от Лондона, причинивший огромный ущерб и сопро­вождавшийся человеческими жертвами? И разве он, премьер-министр, виноват в том, что один из его заместителей, Герберт Чорли, именно на этой не­деле начал вести себя так своеобразно, что ему те­перь придется значительно больше времени про­водить дома, с семьей? «Страну охватило уныние», — закончил свою речь представитель оппозиции, почти не скрывая широ­кой довольной улыбки. Увы, тут он сказал чистую правду. Премьер-ми­нистр и сам это почувствовал: люди выглядели не­привычно подавленными. Даже погода стояла без­радостная. Промозглый туман в середине июля... Не­правильно это. Ненормально.'''

In [ ]:
analysis = aggregate_info(text)
analysis.head()

,token,lemma,pos,stop
0,приближалась,приближаться,VERB,False
1,полночь,полночь,NOUN,False
2,.,.,PUNCT,False
3,премьер,премьер,NOUN,False
4,-,-,NOUN,False


In [ ]:
word_freq = form_freq_df(analysis)
word_freq.head()

,token,lemma,pos,stop,count
0,",",",",PUNCT,False,29
1,.,.,PUNCT,False,16
2,в,в,ADP,True,9
3,и,и,CCONJ,True,9
4,на,на,ADP,True,6


In [ ]:
top = get_top_freq_words(word_freq, 'NOUN')
top.head()

,token,lemma,pos,stop,count
5,-,-,NOUN,False,6
7,премьер,премьер,NOUN,False,5
13,министр,министр,NOUN,False,3
18,правительство,правительство,NOUN,False,2
27,образом,образ,NOUN,False,2


Как можно заметить, топ слов по каждой из проверенных частей речи примерно совпадает что в русском, что в английском. 

In [ ]:
print(get_top_freq_words(word_freq, 'VERB').head(10)['token'].to_list())
print(get_top_freq_words(word_freq, 'ADV').head(10)['token'].to_list())
print(get_top_freq_words(word_freq, 'ADJ').head(10)['token'].to_list())

['лежала', 'обязано', 'тратится', 'разломился', 'перечислил', 'охватило', 'отправив', 'отделаться', 'оставалось', 'освещавшихся']
['еще', 'когда', 'почти', 'одновременно', 'отчего', 'отчет\xadливее', 'подробно', 'тут', 'увы', 'широко']
['ненормально', 'ужасные', 'далекой', 'своеобразно', 'сам', 'чистую', 'человеческими', 'утомительной', 'подавленными', 'лучшие']
